In [1]:
%load_ext autoreload
%autoreload 2
import argparse
import os 
from habitat_sim.agent.controls.controls import SceneNodeControl
import open3d as o3d
import numpy as np
from dataset.habitat.simulator import init_sim 
from plyfile import PlyData, PlyElement

# local import 
from scene_graph.scene_graph_cls import SceneGraphHabitat
from scene_graph.config import SceneGraphHabitatConfig
from scene_graph.utils import visualize_scene_graph, getOBB
from scene_graph.scene_graph_pred import SceneGraphPredictor

def parse_args(input=""):

    parser = argparse.ArgumentParser()
    parser.add_argument("--scan_dir", type=str, default="/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans")
    parser.add_argument("--scene_name", type=str, default="17DRP5sb8fy")
    parser.add_argument("--dataset", type=str, default="matterport")
    
    parser.add_argument("--visualize", type=int, default=1)
    parser.add_argument("--vis_mode", type=str, default="segment", choices=["rgb", "segment"])
    
    parser.add_argument("--rel_dist_thresh", type=float, default=2.0, help="threshold of max distance between two objects that could have a relationship")

    args = parser.parse_args(input)  
    args.scene_ply_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}_semantic.ply')
    args.scene_glb_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}.glb')
    args.pclseg_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}_pclseg.txt')
    args.pcl_normals_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}_normals.npy')
    args.house_file_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}.house')
    return args 




SyntaxError: invalid syntax (scene_graph_pred.py, line 115)

In [ ]:
args = parse_args()
############ load ground truth pointclouds ####################
# o3d_pcl = o3d.io.read_point_cloud(args.scene_ply_path) # only load vertices 
# scene = o3d.io.read_triangle_mesh(args.scene_ply_path) # load mesh file \


In [ ]:
############ initialize habitat simulator and ground truth scene graph ########
sim, action_names, sim_settings = init_sim(args.scene_glb_path)

I1125 15:49:39.141994 42515 ManagedFileBasedContainer.h:210] <Dataset>::convertFilenameToPassedExt : Filename : default changed to proposed scene_dataset_config.json filename : default.scene_dataset_config.json
I1125 15:49:39.142017 42515 AttributesManagerBase.h:365] <Dataset>::createFromJsonOrDefaultInternal : Proposing JSON name : default.scene_dataset_config.json from original name : default | This file  does not exist.
I1125 15:49:42.677098 42515 simulator.py:221] Loaded navmesh /media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/17DRP5sb8fy/17DRP5sb8fy.navmesh
I1125 15:49:39.142112 42515 AssetAttributesManager.cpp:120] Asset attributes (capsule3DSolid : capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
I1125 15:49:39.142138 42515 AssetAttributesManager.cpp:120] Asset attributes (capsule3DWireframe : capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
I1125 

Renderer: GeForce RTX 3080/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 460.91.03
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_ES2_compatibility
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
Using driver workarounds:
    no-forward-compatible-core-context
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits
agent_state: position [0. 0. 0.] rotation quaternion(1, 0, 0, 0)
sensor_states: {'rgb': SixDOFPose(position=array([0. , 1.5, 0. ], dtype=float32), rotation=quaternion(1, 0, 0, 0)), 'depth': SixDOFPose

frame) created and registered.
I1125 15:49:39.142204 42515 AssetAttributesManager.cpp:120] Asset attributes (cylinderSolid : cylinderSolid_rings_1_segments_12_halfLen_1_useTexCoords_false_useTangents_false_capEnds_true) created and registered.
I1125 15:49:39.142220 42515 AssetAttributesManager.cpp:120] Asset attributes (cylinderWireframe : cylinderWireframe_rings_1_segments_32_halfLen_1) created and registered.
I1125 15:49:39.142228 42515 AssetAttributesManager.cpp:120] Asset attributes (icosphereSolid : icosphereSolid_subdivs_1) created and registered.
I1125 15:49:39.142235 42515 AssetAttributesManager.cpp:120] Asset attributes (icosphereWireframe : icosphereWireframe_subdivs_1) created and registered.
I1125 15:49:39.142246 42515 AssetAttributesManager.cpp:120] Asset attributes (uvSphereSolid : uvSphereSolid_rings_8_segments_16_useTexCoords_false_useTangents_false) created and registered.
I1125 15:49:39.142257 42515 AssetAttributesManager.cpp:120] Asset attributes (uvSphereWireframe :

In [ ]:
%reload_ext autoreload
import sys
from importlib import reload

# del sys.modules['scene_graph.scene_graph_cls'] 
from scene_graph import scene_graph_cls
from scene_graph.config import SceneGraphHabitatConfig

# intialize ground truth scene graph 
config = SceneGraphHabitatConfig()
sg = scene_graph_cls.SceneGraphHabitat(config, scene_name=args.scene_name)
sg.load_gt_scene_graph(args.scene_ply_path, args.pclseg_path, args.pcl_normals_path, sim) 

 10%|█         | 1/10 [00:00<00:01,  8.78it/s]

100%|██████████| 10/10 [00:00<00:00, 10.95it/s]


In [ ]:
# %reload_ext autoreload
# from scene_graph.utils import visualize_scene_graph 
# ########### visualize loaded scene with bounding boxes ########################
# if args.visualize:
#     visualize_scene_graph(sg, free_space_vis=True, house_file_path=args.house_file_path)


## Test the scene graph predcition generated by 3DSSG

In [ ]:
from dataset.habitat.utils import read_house_file
house_dict, meta_dict = read_house_file(args.house_file_path)

In [ ]:
############ extract GCN features by pretrained 3DSSG model 
feature_extractor = SceneGraphPredictor(args.rel_dist_thresh)
object_nodes = [sg.object_layer.obj_dict[obj_id]
    for obj_id in sg.object_layer.obj_ids]

''' 
extractor returns a dictionary:  
results={
    "pred_obj_prob": pred_obj_prob, # (N, D) numpy array
    "pred_obj_confidence": pred_obj_confidence, # (N,) numpy array
    "pred_obj_label": pred_obj_label, # (N,) numpy array 
    "edges": edges, # (M,2) numpy array, represented by object index (not id!)
    "pred_rel_prob": pred_rel_prob, # (M,2)
    "pred_rel_confidence": pred_rel_confidence,
    "pred_rel_label": pred_rel_label
}
'''
# object center, bounding box, 
results = feature_extractor.predict(object_nodes)

==trainable parameters==
obj_encoder 73976
rel_encoder 42112
gcn 5274368
obj_predictor 399124
rel_predictor 265225


Loading CVPR21 model...
	No checkpoints, but has saved best model. Load the best model
	model loaded!



## Test object prediction accuracy 

In [ ]:
# load gt label mapping 
from SSG.utils.util_label import getLabelNameMapping, getLabelMapping
from SSG.utils import define as ssg_def
# 1. load mapping from scannet20 to nyu40 
# since scannet 20 is a proper subset of nyu40, only mapping index works    
getLabelMapping("scannet20", ssg_def.LABEL_MAPPING_FILE)
